<a href="https://colab.research.google.com/github/axelmedjber/CarbonFootprintApp/blob/main/Copy_of_Used_Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e9:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76728%2F9057646%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240927%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240927T184013Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dadd7ee7604cb8ceb710c9171fae152da44b9a77bfdc5024170179cf107e6e3c009cba46c4a3ffddd6a1e97b03da7bd1e25e81f9b07345e68154fb07fe453950131718ed45e3496cff63b056a14e28e1d78cdf61abc22a824123e205e0235fef1847361c3ddd7362c97b22c8a6052003a5a3ee81dfaff1e839b36b12c1b3b9861aafd235f05f5c17bfe28d09ddd89858e3b472c74ae1fac1ee977f0dfb2934ad4298cdfcc33e3b36af67baaf162ca1cc61329bc3665eafb77d1a3c22a9e27e05c9abc8b64f6f240e41f1fe7fdae2946908524bd0f6d058af4166d2a374266fc51e9934f6067295eb31aca11dcc253af038d9bd891776636fb05f8e9299ed24a7e,car-price:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3597439%2F6259095%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240927%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240927T184013Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D53e9ec6c589af30d60b31bfe076abbfa41c9575bb187198bcf7a41dbcc70150c77e733e901c2a8aec2ab321aff38f649ef8eb9551fdf55fc4d1ff5057be5cd2796c5d5a8556b9b75f07df0a20f073ab52a3b2605a678cc145f1cf18fcc594eb2080bdd915aed6c03902ba1be335efa8125ac6528424605c23942a08d792c0293d2a34fe1d3c2a9a324c1582993e025be29e1b081fe6f4f003c6b9fb89fbcbf20d354f6c1441b1e92c6939bf6b5078bfa1cab3ca7e5e020b85a7604aca3b5bb77e95c802d03d783a63f96a874485bc8d1cb2db685f3c65306eb5f78f198c3aa36ba166c75b7d7e7246bdb21e1ebcf984d5db8e447544af6d34c034cb7ae6fe216,used-car-price-prediction-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3742543%2F6478229%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240927%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240927T184013Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D85af8f0c0fe0ab7972d4e46c02310bbebc70213a3f8c3f9991ad7a892b9bdfa533e9a065ae2b4f02f8a60200d322c7816f00d8451b3af816625c46103f8eacbbc2352d35ac034290ed9805afd4e2f8da97c9225d8cc48b14d26c773875e8efd73df82b277d5e9f6238f663a71df571147b021a48446bcf45240b036154c75d4061ef8d6d09b0cabe9549fcd2892d73dde44cef8def201b238582ccf365159bbd8bd498777274f77651e5a0efc0831dcf46151e59a507abd17b903477c4ecab1be1c535cbe8c68b85ea8776b1e9daff4fe781ea7d3f64202beead534b812c7aa36f8bc5e06fae2f61072cb8452864f461d61dd148ba5d3d9d85c953432a1cdff5,extended-dataset-for-used-car-prices-regression:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5664146%2F9345432%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240927%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240927T184013Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D060c35b9ceee36fd780590c2f5962455374e3ea380bd88f924b367433000d1e1031875202a348699135aa875cee768cf34e67a5e7b64ab3f6bf722fa4d2f39e45ac99ab535ac9d6f9a3e69f25e2b93147f449fbe2484b9ca04399514dc0dea569488168e9baafcb5b1ec0afc8af2cbd1d47f4f8653dee6618e277f041b67ab0128abb75ed3c1fdd2adf46585c4ac08a3a89743fd3b464fac85f0583c06865cd5a09b32259cf87107f5c00af634de7f56f02b13025b3cc56b1e1278039fed2fb9ccc24ed0c8ff8a3fa6d470154c350e1ce0e864e066f5de2969bdf3fa6e1be227f03ddfa0a46ad4865ccdf12a2df488e58819a280defd3c63d44138d870662de5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 8223535 bytes downloaded
Downloaded and uncompressed: playground-series-s4e9
[==================================================] 18523 bytes downloaded
Downloaded and uncompressed: car-price
[==================================================] 112006 bytes downloaded
Downloaded and uncompressed: used-car-price-prediction-dataset
[==================================================] 332221 bytes downloaded
Downloaded and uncompressed: extended-dataset-for-used-car-prices-regression
Data source import complete.


# **Car Price Prediction: A Machine Learning Approach**

In this notebook, we aim to predict the prices of used cars using a range of advanced machine learning techniques. The dataset includes various features such as brand, model, engine specifications, and mileage, which are used to estimate the final price.

The approach involves:
- Extensive **feature engineering** to enhance model performance.
- **Hyperparameter tuning** to optimize model parameters for better accuracy.
- **Ensemble learning** to combine the predictions of multiple models (XGBoost, LightGBM, CatBoost, and Random Forest).
- **GPU memory management** and optimization techniques to handle large datasets efficiently.

## **1. Import Libraries**

In [ ]:
# Install necessary libraries quietly
!pip install --upgrade pip >/dev/null
!pip install optuna xgboost lightgbm catboost category_encoders >/dev/null

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import optuna
import re
import sys
import os
import contextlib

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Import enable_iterative_imputer before using IterativeImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

import category_encoders as ce

import warnings
warnings.filterwarnings('ignore')

# Suppress Optuna logging
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Context manager to suppress stdout and stderr
@contextlib.contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## **2. Load Data**

In [ ]:
df_sub = pd.read_csv("/kaggle/input/playground-series-s4e9/sample_submission.csv")
df_train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")

## **3. Exploratory Data Analysis (EDA)**

In [ ]:
# Display basic information
print("Train Data Shape:", df_train.shape)
print("Test Data Shape:", df_test.shape)
display(df_train.head())

# Check for missing values
missing_values = df_train.isnull().sum()
print("\nMissing Values in Train Data:\n", missing_values)

Train Data Shape: (188533, 13)
Test Data Shape: (125690, 12)


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500



Missing Values in Train Data:
 id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64


## **4. Data Preprocessing**

In [ ]:
# Combine train and test data for consistent preprocessing
df_train['is_train'] = 1
df_test['is_train'] = 0
df = pd.concat([df_train, df_test], ignore_index=True)

# Drop 'id' column if present
df.drop(columns=['id'], errors='ignore', inplace=True)

# Handle missing values using Iterative Imputer for numerical features
def iterative_impute(df):
    imputer = IterativeImputer(random_state=42)
    numeric_cols = df.select_dtypes(exclude=['object']).columns.drop('is_train', errors='ignore')
    df_numeric = df[numeric_cols]
    df_imputed = pd.DataFrame(imputer.fit_transform(df_numeric), columns=numeric_cols)
    df.update(df_imputed)
    return df

df = iterative_impute(df)

## **5. Clean 'engine' Column**

In [ ]:
def clean_engine(engine_value):
    if isinstance(engine_value, str):
        # Extract numeric part
        match = re.search(r'([\d\.]+)', engine_value)
        if match:
            return float(match.group(1))
        else:
            return np.nan
    else:
        return engine_value

df['engine'] = df['engine'].apply(clean_engine)

# Handle any remaining NaN values in 'engine' after cleaning
df['engine'] = df['engine'].fillna(df['engine'].mean())

## **6. Feature Engineering**

In [ ]:
current_year = 2024
luxury_brands = ['Mercedes-Benz', 'BMW', 'Audi', 'Lexus', 'Tesla', 'Porsche', 'Jaguar', 'Land Rover']

def create_features(df):
    df['vehicle_age'] = current_year - df['model_year']
    df['mileage_per_year'] = df['milage'] / (df['vehicle_age'] + 1)
    df['rare_fuel_type'] = df['fuel_type'].apply(lambda x: 0 if x in ['Petrol', 'Diesel'] else 1)
    df['is_automatic'] = df['transmission'].apply(lambda x: 1 if x == 'Automatic' else 0)
    df['has_accident_history'] = df['accident'].apply(lambda x: 1 if x != 'Unknown' and x != 'None' else 0)

    # Create new features based on existing ones
    df['engine_power'] = df['engine'] * df.get('horsepower', df['engine'])

    # Handle 'weight' column if it exists
    if 'weight' in df.columns:
        df['power_to_weight'] = df.get('horsepower', df['engine']) / (df['weight'] + 1)
    else:
        # If 'weight' is not available, use 'engine' as a proxy
        df['power_to_weight'] = df.get('horsepower', df['engine']) / (df['engine'] + 1)

    df['brand_model'] = df['brand'] + '_' + df['model']
    df['is_luxury_brand'] = df['brand'].isin(luxury_brands).astype(int)
    df['engine_transmission'] = df['engine'] * df['is_automatic']
    df['age_mileage'] = df['vehicle_age'] * df['milage']

    return df

# Apply feature engineering
df = create_features(df)

### **a. Clustering**

In [ ]:
# Use KMeans clustering on numerical features to create cluster labels
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop(['price', 'is_train'], errors='ignore')
kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
df['cluster'] = kmeans.fit_predict(df[numeric_cols])

### **b. Polynomial Features**

In [ ]:
# Generate polynomial and interaction features
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_features = poly.fit_transform(df[numeric_cols])
poly_feature_names = poly.get_feature_names_out(numeric_cols)
df_poly = pd.DataFrame(poly_features, columns=poly_feature_names)

# Exclude original features from df_poly
original_features_set = set(numeric_cols)
df_poly = df_poly[[col for col in df_poly.columns if col not in original_features_set]]

# Concatenate the new features
df.reset_index(drop=True, inplace=True)
df_poly.reset_index(drop=True, inplace=True)
df = pd.concat([df, df_poly], axis=1)

### **c. Target Encoding with Cross-Validation**

In [ ]:
# Define high cardinality categorical columns
high_cardinality_cols = ['brand', 'model', 'brand_model', 'ext_col', 'int_col', 'fuel_type', 'transmission', 'accident']

# Prepare data for target encoding
train_df = df[df['is_train'] == 1]
test_df = df[df['is_train'] == 0]

# Initialize TargetEncoder with smoothing to prevent overfitting
target_enc = ce.TargetEncoder(cols=high_cardinality_cols, smoothing=0.3)

# Apply cross-validated target encoding to avoid data leakage
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_encoded = train_df.copy()

for train_index, val_index in kf.split(train_df):
    X_train, X_val = train_df.iloc[train_index], train_df.iloc[val_index]
    y_train = X_train['price']
    target_enc.fit(X_train[high_cardinality_cols], y_train)
    train_encoded.iloc[val_index, train_encoded.columns.get_indexer(high_cardinality_cols)] = target_enc.transform(X_val[high_cardinality_cols])

# Fit encoder on the entire training data and transform test data
target_enc.fit(train_df[high_cardinality_cols], train_df['price'])
test_encoded = test_df.copy()
test_encoded[high_cardinality_cols] = target_enc.transform(test_df[high_cardinality_cols])

# Combine the encoded data
df = pd.concat([train_encoded, test_encoded], axis=0)

## **7. Handle Outliers and Skewness**

In [ ]:
# Identify skewed numerical features
from scipy.stats import skew

numeric_feats = df.select_dtypes(include=['float64', 'int64']).columns.drop(['price', 'is_train'], errors='ignore')
skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew': skewed_feats})

# Apply log1p transformation to features with high skewness
skewed_features = skewness[abs(skewness['Skew']) > 0.75].index
df[skewed_features] = np.log1p(df[skewed_features])

# Apply log transformation to the target variable
df['price'] = np.log1p(df['price'])

## **8. Encoding Remaining Categorical Variables**

In [ ]:
# Encode remaining categorical variables using OrdinalEncoder
cat_cols = df.select_dtypes(include=['object']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[cat_cols] = ordinal_encoder.fit_transform(df[cat_cols].astype(str))

## **9. Scaling Numerical Features**

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop(['price', 'is_train'], errors='ignore')
scaler = RobustScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

## **10. Feature Selection**

In [ ]:
# Remove features with low variance
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
selector.fit(df[df['is_train'] == 1][numeric_cols])
low_variance_cols = numeric_cols[~selector.get_support()]
print("Low Variance Columns:", low_variance_cols.tolist())

df.drop(columns=low_variance_cols, inplace=True)

# Update numeric_cols after feature selection
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop(['price', 'is_train'], errors='ignore')

Low Variance Columns: ['rare_fuel_type', 'has_accident_history', 'rare_fuel_type has_accident_history']


## **11. Prepare Data for Modeling**

In [ ]:
# Split back into train and test sets
df_train = df[df['is_train'] == 1].drop(columns=['is_train'])
df_test = df[df['is_train'] == 0].drop(columns=['is_train', 'price'])

y = df_train['price']
X = df_train.drop(['price'], axis=1)

# Ensure no duplicate columns
X = X.loc[:, ~X.columns.duplicated()]
df_test = df_test.loc[:, ~df_test.columns.duplicated()]

print("Final Training Features Shape:", X.shape)
print("Final Test Features Shape:", df_test.shape)

Final Training Features Shape: (188533, 98)
Final Test Features Shape: (125690, 98)


## **12. Hyperparameter Tuning with Optuna**

In [ ]:
def objective_xgb(trial):
    # Define the parameters to be optimized
    params = {
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'random_state': 42,
        # Use CPU for training
        'tree_method': 'hist',
        'verbosity': 0  # Suppress messages
    }

    # Use cross-validation
    model = XGBRegressor(**params)
    with suppress_stdout_stderr():
        scores = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    return -scores.mean()

# Optuna hyperparameter tuning for XGBoost
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=10)

best_params_xgb = study_xgb.best_params
print("Best XGBoost Params:", best_params_xgb)

Best XGBoost Params: {'subsample': 0.905081117520834, 'learning_rate': 0.03111589583720229, 'max_depth': 8, 'min_child_weight': 28, 'n_estimators': 381}


## **13. Training Final Models**

### **a. XGBoost**

In [ ]:
final_xgb = XGBRegressor(**best_params_xgb)
with suppress_stdout_stderr():
    final_xgb.fit(X, y)

### **b. LightGBM**

In [ ]:
def objective_lgb(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'random_state': 42,
        # 'device': 'gpu',  # Remove this line to let LightGBM choose the device
        'verbosity': -1,  # Suppress messages
    }

    model = lgb.LGBMRegressor(**params)
    with suppress_stdout_stderr():
        scores = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    return -scores.mean()

# Optuna hyperparameter tuning for LightGBM
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=10)

best_params_lgb = study_lgb.best_params
print("Best LightGBM Params:", best_params_lgb)

Best LightGBM Params: {'num_leaves': 132, 'max_depth': 15, 'learning_rate': 0.011169919228287292, 'n_estimators': 640, 'min_data_in_leaf': 50, 'feature_fraction': 0.7261264051360649, 'bagging_fraction': 0.6187112228344223, 'bagging_freq': 6, 'lambda_l1': 1.3979535589830219, 'lambda_l2': 0.3763764272233558}


In [ ]:
# Remove 'verbose' from best_params_lgb if present
best_params_lgb.pop('verbose', None)
best_params_lgb.pop('verbosity', None)  # Remove if causing issues

# Initialize the model with 'verbose' parameter
final_lgb = lgb.LGBMRegressor(**best_params_lgb, verbose=-1)

# Train the model while suppressing output
with suppress_stdout_stderr():
    final_lgb.fit(X, y)

### **c. CatBoost**

In [ ]:
def objective_cat(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'eval_metric': 'RMSE',
        'random_seed': 42,
        'task_type': 'GPU',
        'verbose': 0  # Suppress messages
    }

    model = CatBoostRegressor(**params)
    with suppress_stdout_stderr():
        # Set n_jobs=1 to prevent multiple processes from using GPU simultaneously
        scores = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error', n_jobs=1)
    return -scores.mean()

# Optuna hyperparameter tuning for CatBoost
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=10)

best_params_cat = study_cat.best_params
print("Best CatBoost Params:", best_params_cat)

Best CatBoost Params: {'iterations': 965, 'depth': 10, 'learning_rate': 0.024098748193505836, 'l2_leaf_reg': 6.965313810356091, 'random_strength': 0.71188566813244, 'bagging_temperature': 0.12392665275739945}


In [ ]:
final_cat = CatBoostRegressor(**best_params_cat)
with suppress_stdout_stderr():
    final_cat.fit(X, y)

### **d. Random Forest**

In [ ]:
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_float('max_features', 0.5, 1.0),
        'random_state': 42,
        'n_jobs': -1,
    }

    model = RandomForestRegressor(**params)
    with suppress_stdout_stderr():
        scores = cross_val_score(model, X, y, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    return -scores.mean()

# Optuna hyperparameter tuning for Random Forest
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=10)

best_params_rf = study_rf.best_params
print("Best Random Forest Params:", best_params_rf)

In [ ]:
final_rf = RandomForestRegressor(**best_params_rf)
with suppress_stdout_stderr():
    final_rf.fit(X, y)

## **14. Stacking Ensemble**

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('xgb', final_xgb),
    ('lgb', final_lgb),
    ('cat', final_cat),
    ('rf', final_rf)
]

final_estimator = Ridge(alpha=1.0)

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=final_estimator,
    cv=3,
    n_jobs=-1
)

with suppress_stdout_stderr():
    stacking_regressor.fit(X, y)

## **15. Predictions**

In [ ]:
# Predict on test data
test_pred_stack = stacking_regressor.predict(df_test)

# Inverse log transformation
final_test_pred = np.expm1(test_pred_stack)

## **16. Create Submission File**

In [ ]:
df_sub['price'] = final_test_pred
df_sub.to_csv('submission.csv', index=False)
display(df_sub.head())

## **17. Feature Importance**

In [ ]:
def plot_feature_importance(model, model_name):
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1][:20]
        plt.figure(figsize=(12, 6))
        plt.title(f"Top 20 Feature Importances - {model_name}")
        plt.bar(range(len(indices)), importances[indices], color='b', align='center')
        plt.xticks(range(len(indices)), [X.columns[i] for i in indices], rotation=90)
        plt.tight_layout()
        plt.show()
    else:
        print(f"{model_name} does not support feature importance.")

In [ ]:
plot_feature_importance(final_xgb, 'XGBoost')

In [ ]:
plot_feature_importance(final_lgb, 'LightGBM')

In [ ]:
plot_feature_importance(final_cat, 'CatBoost')

# **Conclusion:**

In this project, a comprehensive machine learning pipeline was developed to predict used car prices. Multiple optimization techniques were implemented to enhance model performance and address challenges related to computational efficiency and resource constraints.

### **1. Feature Engineering and Selection:**
- A wide range of feature engineering techniques were applied, including interaction terms (e.g., mileage, vehicle age, and engine specifications), allowing the model to capture key patterns within the data.
- Feature importance analysis from XGBoost, LightGBM, and CatBoost helped identify the most relevant features, reducing the feature space while retaining high predictive power.

### **2. Handling Missing Data and Imbalanced Features:**
- Missing values were handled using **Iterative Imputer**, while categorical variables were encoded using strategies such as target and ordinal encoding. Rare or imbalanced categories (e.g., fuel type) were carefully managed to prevent model bias.

### **3. Model Optimization:**
- Models, including XGBoost, LightGBM, and CatBoost, were optimized using **Optuna** for hyperparameter tuning. This ensured the models were configured to perform optimally while balancing computational load.
- GPU memory errors were addressed by adjusting model parameters (e.g., limiting tree depth, adjusting batch sizes) and, where necessary, switching to CPU-based operations.

### **4. Hyperparameter Tuning Efficiency:**
- Hyperparameter tuning was made more efficient by narrowing the search space based on previous experiments. To speed up the process, **cross-validation (cv=3)** was used, balancing between computation and performance estimation.
- Early stopping was applied during model training to prevent overfitting and reduce training time.

### **5. Managing GPU Memory Constraints:**
- Memory issues, particularly GPU out-of-memory (OOM) errors, were mitigated by optimizing batch sizes and model parameters. Processes were moved to the CPU when GPU memory was exhausted.

### **6. Ensemble Learning:**
- A **stacking ensemble** was implemented to combine the strengths of XGBoost, LightGBM, and CatBoost. This ensemble approach leveraged the diverse patterns captured by each model, leading to improved overall performance.

### **7. Final Results:**
- Through careful feature selection, hyperparameter tuning, and ensemble modeling, the final model achieved significant improvements in predictive accuracy. The combination of memory optimization techniques and efficient cross-validation helped to balance model complexity with computational resources.

---

This project successfully demonstrated the importance of balancing model complexity, feature selection, and computational efficiency to build a robust price prediction model. By addressing the challenges of GPU memory usage and model tuning, we were able to deliver a solution that provides accurate predictions while remaining computationally feasible.